#### Fardin Rastakhiz @2023

In [1]:
import numpy as np
from Scripts.Configs.ConfigClass import Config
from Scripts.DataManager.GraphConstructor.GraphConstructor import TextGraphType
from lightning.pytorch.loggers import CSVLogger
import os
from Scripts.DataManager.GraphLoader.AmazonReviewGraphDataModule import AmazonReviewGraphDataModule
import time
import torch

c:\Users\fardin\AppData\Local\Programs\Python\Python310\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
config = Config(r'C:\Users\fardin\Projects\ColorIntelligence')
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
device = 'cuda'
batch_size = 256

In [3]:
# tag_dep_seq_sent = TextGraphType.TAGS | TextGraphType.DEPENDENCY | TextGraphType.SEQUENTIAL | TextGraphType.SENTENCE | TextGraphType.SENTIMENT
tag_dep_seq_sent = TextGraphType.SENTIMENT
data_manager = AmazonReviewGraphDataModule(config, True, True, shuffle=False, start_data_load=0 , end_data_load = 12000, device='cpu', batch_size=batch_size, graph_type=tag_dep_seq_sent, load_preprocessed_data = True)
data_manager.load_labels()
data_manager.load_graphs()

filename: C:\Users\fardin\Projects\ColorIntelligence\data/GraphData/AmazonReview\sentiment\graph_var.txt


 Loding Graphs From File : 100%|██████████| 12/12 [02:07<00:00, 10.64s/it]

self.shuffle: False


In [4]:
# data_manager.update_batch_size(128)
t_dataloader = data_manager.train_dataloader()
v_dataloader = data_manager.val_dataloader()
X1, y1 = next(iter(t_dataloader))
X2, y2 = next(iter(v_dataloader))
# X1.metadata()
len(X1)

256

In [16]:
X1.metadata()

(['dep', 'tag', 'word', 'sentence', 'general', 'sentiment'],
 [('dep', 'dep_word', 'word'),
  ('word', 'word_dep', 'dep'),
  ('tag', 'tag_word', 'word'),
  ('word', 'word_tag', 'tag'),
  ('word', 'seq', 'word'),
  ('general', 'general_sentence', 'sentence'),
  ('sentence', 'sentence_general', 'general'),
  ('word', 'word_sentence', 'sentence'),
  ('sentence', 'sentence_word', 'word'),
  ('word', 'word_sentiment', 'sentiment'),
  ('sentiment', 'sentiment_word', 'word')])

In [8]:
for i, k in enumerate(X1.metadata()[0]):
    print(f'i: {i}, k: {k}')

i: 0, k: dep
i: 1, k: tag
i: 2, k: word
i: 3, k: sentence
i: 4, k: general
i: 5, k: sentiment


In [9]:
from Scripts.Models.GraphEmbedding.HeteroDeepGraphEmbedding3 import HeteroDeepGraphEmbedding3
from Scripts.Models.LightningModels.LightningModels import HeteroBinaryLightningModel
from Scripts.Models.LossFunctions.HeteroLossFunctions import HeteroLossArgs, HeteroLoss1, HeteroLoss2
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from Scripts.Models.ModelsManager.ClassifierModelManager import ClassifierModelManager

In [119]:

import torch.nn.functional as F
from torch import Tensor
import torch
from torch.nn import Linear
from torch_geometric.nn import BatchNorm, MemPooling, to_hetero, PairNorm
from torch_geometric.data import HeteroData
from Scripts.Models.BaseModels.HeteroGat import HeteroGat
from Scripts.Models.BaseModels.HeteroLinear import HeteroLinear
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv


class HeteroDeepGraphEmbedding3(torch.nn.Module):
    
    def __init__(self,
                 input_feature: int, out_features: int,
                 metadata,
                 hidden_feature: int=256,
                 device = 'cpu',
                 dropout=0.1,
                 edge_type_count=9,
                 edge_type_weights=-1,
                 pivot_node='word'
                 ):

        super(HeteroDeepGraphEmbedding3, self).__init__()
        self.input_features = input_feature
        self.num_out_features = out_features
        self.bsh: int = hidden_feature
        self.edge_type_count = edge_type_count
        self.metadata = metadata
        self.pivot_node = pivot_node
        self.edge_type_weights = [1] * self.edge_type_count if edge_type_weights==-1 else edge_type_weights

        self.part_weight_norm = torch.nn.LayerNorm((self.edge_type_count,))
        self.norm = PairNorm()
        self.drop = torch.nn.Dropout(0.2)
        self.hetero_linear_1 = to_hetero(HeteroLinear(input_feature, self.bsh, dropout), self.metadata)
        
        self.hetero_gat_1 = to_hetero(HeteroGat(self.bsh, self.bsh, dropout, num_heads=2), self.metadata)
        self.hetero_gat_2 = to_hetero(HeteroGat(self.bsh, self.bsh, dropout, num_heads=2), self.metadata)
        self.hetero_gat_3 = to_hetero(HeteroGat(self.bsh, self.bsh, dropout, num_heads=2), self.metadata)
        
        self.hetero_linear_2 = to_hetero(HeteroLinear(self.bsh, input_feature, dropout, use_batch_norm=True), self.metadata)
        self.hetero_linear_3 = to_hetero(HeteroLinear(input_feature, input_feature, dropout, use_dropout=False), self.metadata)
        
        self.mem_pools = []
        for i in range(len(self.metadata[0])):
            self.mem_pools.append(MemPooling(self.bsh, self.bsh, 2, 1))
        self.mem_pools = torch.nn.ModuleList(self.mem_pools)
        
        self.conv1 = GCNConv(self.bsh, self.bsh)
        
        self.mem_pool_2 = MemPooling(self.bsh, self.bsh, 2, 1)
        
        # self.linear_1_list = []
        # for i in range(len(self.metadata[0])):
        #     self.linear_1_list.append(Linear(self.num_out_features * self.bsh, self.bsh))
        # self.linear_1_list = torch.nn.ModuleList(self.linear_1_list)
        
        
        # self.type_aggregation = torch.nn.GRU()
        
        
        
        self.linear_1 = Linear(self.bsh, self.bsh)
        self.linear_2 = Linear(self.bsh, self.bsh)
        self.linear_3 = Linear(self.bsh, 64)
        self.batch_norm_1 = BatchNorm(64)
        
        self.output_layer = Linear(64, self.num_out_features)

        self.dep_embedding = torch.nn.Embedding(45, self.input_features)
        self.tag_embedding = torch.nn.Embedding(50, self.input_features)
        self.dep_unembedding = torch.nn.Linear(self.input_features, 45)
        self.tag_unembedding = torch.nn.Linear(self.input_features, 50)
        
        self.pw1 = torch.nn.Parameter(torch.tensor(self.edge_type_weights, dtype=torch.float32), requires_grad=False)
        self.pw2 = torch.nn.Parameter(torch.tensor(self.edge_type_weights, dtype=torch.float32), requires_grad=False)
        
        self.node_types = {n for i, e in enumerate(metadata[1]) for n in {e[0], e[2]} if self.edge_type_weights[i]!=0}

    def forward(self, x: HeteroData) -> Tensor:
        x_dict, edge_attr_dict, edge_index_dict = self.preprocess_data(x)
        edge_attr_dict = self.update_weights(edge_attr_dict, self.pw1)
        
        x_dict = self.hetero_linear_1(x_dict)
        
        x_dict = self.hetero_gat_1(x_dict, edge_index_dict, edge_attr_dict)
        x_dict = self.normalize(x_dict, x)
        
        edge_attr_dict = self.update_weights(edge_attr_dict, self.pw2)
        
        x_dict = self.hetero_gat_2(x_dict, edge_index_dict, edge_attr_dict)
        x_dict = self.normalize(x_dict, x)

        x_dict = self.hetero_gat_3(x_dict, edge_index_dict, edge_attr_dict)
        x_pools = []
        pivot_index=0
        j = 0
        for i, k in enumerate(self.metadata[0]):
            if k in self.node_types:
                x_pooled, S = self.mem_pools[i](x_dict[k], x[k].batch)
                x_pooled = torch.unsqueeze(x_pooled.view(x_pooled.shape[0], -1), dim=1)
                x_pools.append(x_pooled)
                if k==self.pivot_node:
                    pivot_index=j
                j += 1
        if len(x_pools) > 1:
            x_pools = torch.concat(x_pools, dim=1) * (len(self.metadata[0]) * 1.0 / len(self.node_types))
            aggr_graph = self.create_aggregated_graph(x_pools, pivot_index, self.pw1.device)
            x_out = self.conv1(aggr_graph.x, aggr_graph.edge_index)
            x_pooled, S = self.mem_pool_2(x_out, aggr_graph.batch)
        else:
            x_pooled = x_pools[0]
            
        x_pooled = x_pooled.view(x_pooled.shape[0], -1)
                    
        x_pooled = F.relu(self.linear_1(x_pooled))
        x_pooled = F.relu(self.linear_2(x_pooled))
        x_pooled = F.relu(self.linear_3(x_pooled))
        x_pooled = F.relu(self.batch_norm_1(x_pooled))
        out = self.output_layer(x_pooled)
        x_dict = self.hetero_linear_2(x_dict)
        x_dict = self.hetero_linear_3(x_dict)
        x_dict['dep'] = self.dep_unembedding(x_dict['dep'])
        x_dict['tag'] = self.tag_unembedding(x_dict['tag'])
        return out, x_dict

    def preprocess_data(self, x):
        x_dict = {key: x.x_dict[key] for key in x.x_dict}
        x_dict['dep'] = self.dep_embedding(x_dict['dep'])
        x_dict['tag'] = self.tag_embedding(x_dict['tag'])

        edge_attr_dict = x.edge_attr_dict
        edge_index_dict = x.edge_index_dict

        shape1 = edge_index_dict[('sentence', 'sentence_word', 'word')].shape[1]
        shape2 = edge_attr_dict[('word', 'word_sentence', 'sentence')].shape[0]
        if shape1 != shape2:
            edge_attr_dict[('sentence', 'sentence_word', 'word')] = edge_attr_dict[('word', 'word_sentence', 'sentence')][shape1:shape2]
            edge_attr_dict[('word', 'word_sentence', 'sentence')] = edge_attr_dict[('word', 'word_sentence', 'sentence')][:shape1]

        # for key in x.edge_attr_dict:
        #     edge_attr_dict[key] = self.get_scale_same(1.0, edge_attr_dict[key])

        return x_dict, edge_attr_dict, edge_index_dict

    def normalize(self, x_dict, x):
        # for key in x_dict:
        #     x_dict[key] = self.norm(x_dict[key], x[key].batch)
        return x_dict

    def update_weights(self, edge_attr_dict, part_weights):
        # part_weights = 0.9 * self.part_weight_norm(part_weights) + 0.05
        # part_weights = 0.9 * F.sigmoid(part_weights) + 0.05
        for i, key in enumerate(edge_attr_dict):
            edge_attr = edge_attr_dict[key]
            if edge_attr is None or edge_attr == ('word', 'seq', 'word'):
                continue
            edge_attr_dict[key]= edge_attr * part_weights[i]
        return edge_attr_dict

    def get_scale_same(self, scale:float, attributes: Tensor):
        if attributes is None or len(attributes) == 0:
            return
        attributes = scale * torch.ones_like(attributes)
        return attributes
    
    def create_aggregated_graph(self, new_nodes: Tensor, pivot_index=0, device= 'cpu'):
        edge_index = torch.tensor([[pivot_index, i] for i in range(new_nodes.shape[1]) if i!=pivot_index], device=device).T
        data_list = []
        for i in range(new_nodes.shape[0]):
            data_graph = Data(x=new_nodes[i], edge_index=edge_index, device=device)
            data_list.append(data_graph)
        batch_graph = Batch.from_data_list(data_list)
        return batch_graph
        
        

In [120]:
graph_embedding = HeteroDeepGraphEmbedding3(300, 1, X1.metadata(), 128, dropout=0.2, edge_type_count=11, edge_type_weights=[1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]).to(device)
graph_embedding(X1.to(device))

(tensor([[-2.9664e-01],
         [ 8.5578e-01],
         [-1.7825e+00],
         [-3.0153e-01],
         [-5.8905e-01],
         [ 6.2493e-01],
         [ 2.8201e-01],
         [ 8.3939e-01],
         [-9.5669e-03],
         [ 9.4852e-02],
         [-5.3513e-01],
         [ 6.1064e-03],
         [ 9.2584e-02],
         [ 1.1942e-01],
         [ 8.1326e-02],
         [ 2.1596e-01],
         [ 2.1799e-01],
         [-1.4275e-01],
         [-1.4546e-01],
         [-5.9504e-01],
         [-8.1713e-01],
         [ 1.7648e-01],
         [-2.1912e-01],
         [-5.0127e-01],
         [-1.8200e-01],
         [-8.9573e-02],
         [ 1.3916e-02],
         [ 3.5024e-01],
         [-1.5335e+00],
         [-8.4542e-02],
         [-1.7846e-01],
         [ 3.0782e-01],
         [-7.2526e-01],
         [-6.7935e-01],
         [-6.4869e-01],
         [-4.3277e-01],
         [-3.4739e-02],
         [-3.8318e-02],
         [-1.4576e-01],
         [-3.6043e-02],
         [-1.0770e-02],
         [ 1.354

In [121]:
import torch
torch.randn((7,256, 128))[:,1,:].shape

torch.Size([7, 128])

In [122]:
edge_type_weights = {
    'seq': [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
    'full': [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
    'dep': [1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
    'tag': [0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
    'general_sentence': [0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0.],
    'sentence': [0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0.],
    'sentiment': [0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.],
}

In [123]:
{n for i, e in enumerate(X1.metadata()[1]) for n in {e[0], e[2]} if edge_type_weights['full'][i]!=0}

{'dep', 'general', 'sentence', 'sentiment', 'tag', 'word'}

In [124]:
for k in edge_type_weights:
    graph_embedding = HeteroDeepGraphEmbedding3(300, 1, X1.metadata(), 128, dropout=0.2, edge_type_count=11, edge_type_weights=edge_type_weights[k])
    graph_embedding = graph_embedding.to(device)
    callbacks = [
    ModelCheckpoint(save_top_k=5, mode='max', monitor='val_acc', save_last=True)
    ]
    lightning_model = HeteroBinaryLightningModel(graph_embedding,
                                    torch.optim.Adam(graph_embedding.parameters(), lr=0.0045, weight_decay=0.001),
                                        loss_func=HeteroLoss1(exception_keys=['word'], enc_factor=0.005),
                                        learning_rate=0.0045,
                                        batch_size=batch_size,
                                        user_lr_scheduler=True,
                                        min_lr=0.0003
                                        ).to(device)
    model_manager = ClassifierModelManager(graph_embedding, lightning_model, log_name='hetero_model_8', model_save_dir=r'C:\Users\fardin\Projects\ColorIntelligence\Practices\Tasks\HeterogeneousGraphs\hetero_model_8',device=device, num_train_epoch=30)
    model_manager.fit(datamodule=data_manager)
    model_manager.save_plot_csv_logger(loss_names=['train_loss', 'val_loss'], eval_names=['train_acc_epoch', 'val_acc_epoch'], name_prepend=f'tests_{k}')
    model_manager.torch_model = model_manager.torch_model.to('cuda')
    model_manager.save_evaluation(data_manager.test_dataloader(), f'{k}',True, True, True, True, True, True, True)

c:\Users\fardin\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\utilities\parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


bbbb



  | Name      | Type                      | Params
--------------------------------------------------------
0 | model     | HeteroDeepGraphEmbedding3 | 2.3 M 
1 | loss_func | HeteroLoss1               | 0     
2 | train_acc | BinaryAccuracy            | 0     
3 | val_acc   | BinaryAccuracy            | 0     
4 | test_acc  | BinaryAccuracy            | 0     
--------------------------------------------------------
2.3 M     Trainable params
22        Non-trainable params
2.3 M     Total params
9.393     Total estimated model params size (MB)
c:\Users\fardin\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\fardin\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\loops\fit_loop.py:293:

Training: |          | 0/? [00:00<?, ?it/s]

c:\Users\fardin\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\call.py:58: Detected KeyboardInterrupt, attempting graceful shutdown...


FileNotFoundError: [Errno 2] No such file or directory: 'logs/hetero_model_8\\version_13\\metrics.csv'